In [1]:
import matplotlib.pyplot as plt
from openml import datasets, tasks, runs, flows, setups
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Find top sklearn.pipeline Flow (with most number of runs)
flowsJson = flows.list_flows()
flowList = pd.DataFrame(flowsJson).transpose()
sklearnPipelineFlows = flowList.loc[flowList.full_name.str.startswith('sklearn.pipeline')]
sklearnRuns = pd.DataFrame(runs.list_runs(flow=sklearnPipelineFlows.id)).transpose()
sklearnRuns.groupby('flow_id').agg('count').sort_values('run_id', ascending = 0)
topFlow = sklearnRuns.groupby('flow_id').agg('count').sort_values('run_id', ascending = 0)[0:1] # flow_id = 5909

OpenMLServerException: Too many results

In [35]:
# Count the number of runs for each flow and task and provide a sample run
topFlowRuns = pd.DataFrame(runs.list_runs(flow=topFlow.index)).transpose()
f = {'run_id':['first','size']}
allRunsTopFlow = topFlowRuns.groupby(['flow_id','task_id']).agg(f).reset_index()
allRunsTopFlow.columns = ['flow', 'task','run', 'count']
allRunsTopFlow['single_execution_time'] = ''

In [ ]:
  # Re-run each flow and task to measure its time
import datetime
for i, row in allRunsTopFlow.iterrows():
    f = flows.get_flow(row.flow)
    t = tasks.get_task(row.task)
    a = datetime.datetime.now()
    runs.run_flow_on_task(t,f)
    b = datetime.datetime.now()
    delta = b - a
    ti = delta.total_seconds()
    print('executed task {}, in {} seconds'.format(i, ti))
    allRunsTopFlow.set_value(i,'single_execution_time', ti)
    #allRunsTopFlow.set_value(i,'time_total', ti * row.count)
allRunsTopFlow.to_csv('top-flow-time.csv')  

In [5]:
allRunsTopFlow.to_csv('top-flow-time.csv')  

NameError: name 'allRunsTopFlow' is not defined

In [6]:
# Read the dataframe from file
df = pd.DataFrame.from_csv('top-flow-time.csv')
df['total_execution_time'] = df['count'] * df['single_execution_time']

- Most popular Flow (pipeline) with id 5909:
- sklearn.pipeline.Pipeline(imputer=sklearn.preprocessing.imputation.Imputer,pca=sklearn.decomposition.pca.PCA,randomforestclassifier=sklearn.ensemble.forest.RandomForestClassifier)
    
    
 

- Executed on 100 unique tasks a total of 897 times
- Sum of repeated execution time on all tasks = 7529 seconds
- Sum of single execution time on all tasks = 839.7 seconds
- Almost an order of magnitude faster

Figure below shows the difference in total time of repeated most popular pipeline (flow 5909) execution vs the a single execution of the pipeline on inidividual tasks. (100 unique tasks wit a total of 897 execution of the pipeline)

In [62]:
sdf = df.sort_values('single_execution_time', ascending=False).reset_index()
del sdf['index']
plt.bar(range(0,100),sdf['single_execution_time'])
plt.bar(range(0,100),sdf['total_execution_time'] - sdf['single_execution_time'], bottom = sdf['single_execution_time'])
#plt.yscale('log')
plt.xlabel('Tasks (sorted by the execution time)', fontsize=18)
plt.ylabel('Execution time (s)', fontsize=16)
plt.title("Total vs Single execution time")
plt.legend(('Single Execution', 'Total Execution'))
#plt.show()

plt.savefig('execution-time.png', bbox_inches='tight', dpi=800)
plt.close()

In [65]:
savings = sdf['total_execution_time'] - sdf['single_execution_time']
sum(savings)

6689.4889910000011